# Robustness test - subsetting DE genes

In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import anndata
from sklearn.linear_model import LogisticRegression
import pickle
import bbknn
import anndata

from scipy.stats import wasserstein_distance
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine

import matplotlib.pyplot as plt
import seaborn as sns
from math import factorial


sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)

scanpy==1.4 anndata==0.6.18 numpy==1.16.2 scipy==1.2.1 pandas==0.24.1 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


### Functions

In [2]:
def create_adatas_dicts(adata,celltype_column):
    adata_dict = {}
    cell_types = list(set(adata.obs[celltype_column]))
    print(len(cell_types),'cell types:', cell_types)
    for i in range(len(cell_types)):
          adata_dict[cell_types[i]] = adata[adata.obs[celltype_column] == cell_types[i]]
    return(adata_dict)

Distance calculating functions

In [3]:
# cosine similarity
def cosine_bw_2_cell_types(adatas_dict_tissue,celltype1,celltype2):
    if list(adatas_dict_tissue[celltype1].var_names) != list(adatas_dict_tissue[celltype2].var_names):
        print('gene lists across cell type tables are not the same')
        return
    else:
        av_vect_ct1 = []
        av_vect_ct2 = []
        for i in range(len(adatas_dict_tissue[celltype1].var_names)):
            #if i%10==0:
            #    print('on gene #',i)
            curr_vect_1 = list(adatas_dict_tissue[celltype1].X[:,i])
            curr_vect_2 = list(adatas_dict_tissue[celltype2].X[:,i])
            av_vect_ct1.append(np.mean(curr_vect_1))
            av_vect_ct2.append(np.mean(curr_vect_2))

        if np.linalg.norm(av_vect_ct1) > 0 and np.linalg.norm(av_vect_ct2) > 0:    # to avoid division by 0
            curr_cosine = cosine(av_vect_ct1,av_vect_ct2)
            return(curr_cosine)
        else:
            print("both specified cell types have expression vectors of zero - don't express any genes at all - returning None, check your data!")
            return(None)

In [4]:
# Wasserstein / Earth mover's distance
def wass_bw_2_cell_types_binned(adatas_dict_tissue,celltype1,celltype2,bin_size=1):
    max_counts = []
    min_counts = []
    if list(adatas_dict_tissue[celltype1].var_names) != list(adatas_dict_tissue[celltype2].var_names):
        print('gene lists across cell type tables are not the same')
        return
    else:
        curr_wass = []
        for i in range(len(adatas_dict_tissue[celltype1].var_names)):
            #if i%10==0:
            #    print('on gene #',i)
            curr_vect_1 = list(adatas_dict_tissue[celltype1].X[:,i])
            curr_vect_2 = list(adatas_dict_tissue[celltype2].X[:,i])
            # maximum count value across the 2 distributions
            curr_max_count = int(max(max(np.unique(adatas_dict_tissue[celltype1].X[:,i])),
                              max(np.unique(adatas_dict_tissue[celltype2].X[:,i]))))
            curr_min_count = int(min(min(np.unique(adatas_dict_tissue[celltype1].X[:,i])),
                              min(np.unique(adatas_dict_tissue[celltype2].X[:,i]))))
            max_counts.append(curr_max_count)
            min_counts.append(curr_min_count)
            #print('current maximum counts',curr_max_count)
            #print('current minimum counts',curr_min_count)
            if np.linalg.norm(curr_vect_1) > 0 and np.linalg.norm(curr_vect_2) > 0:    # to avoid division by 0
                # binning the distributions
                bin_number = int(np.ceil((curr_max_count - curr_min_count)/bin_size))
                #print(bin_number)
                curr_vect_1_binned = list(np.histogram(curr_vect_1,bin_number,density=True)[0])
                curr_vect_2_binned = list(np.histogram(curr_vect_2,bin_number,density=True)[0])
                #print(wasserstein_distance(curr_vect_1_binned,curr_vect_2_binned))
                curr_wass.append(wasserstein_distance(curr_vect_1_binned,curr_vect_2_binned))
        #print('maximum counts was', max(max_counts))
        #print('minimum counts was', min(min_counts))
        av_wass = np.mean(curr_wass)
        return(av_wass)

In [5]:
# Pearson's correlation
def pearson_bw_2_cell_types(adatas_dict_tissue,celltype1,celltype2):
    if list(adatas_dict_tissue[celltype1].var_names) != list(adatas_dict_tissue[celltype2].var_names):
        print('gene lists across cell type tables are not the same')
        return
    else:
        av_vect_ct1 = []
        av_vect_ct2 = []
        for i in range(len(adatas_dict_tissue[celltype1].var_names)):
            #if i%10==0:
            #    print('on gene #',i)
            curr_vect_1 = list(adatas_dict_tissue[celltype1].X[:,i])
            curr_vect_2 = list(adatas_dict_tissue[celltype2].X[:,i])
            av_vect_ct1.append(np.mean(curr_vect_1))
            av_vect_ct2.append(np.mean(curr_vect_2))

        if np.linalg.norm(av_vect_ct1) > 0 and np.linalg.norm(av_vect_ct2) > 0:    # to avoid division by 0
            curr_pearson = pearsonr(av_vect_ct1,av_vect_ct2)
            return(curr_pearson)
        else:
            print("both specified cell types have expression vectors of zero - don't express any genes at all - returning None, check your data!")
            return(None)

Functions creating actual dataframes
<br>
For caution these are separate

In [6]:
def create_df_cosine(adatas_dict_tissue):
    curr_matrix = np.zeros((len(adatas_dict_tissue),len(adatas_dict_tissue)))
    for i in range(len(adatas_dict_tissue)):
        for j in range(i):
            curr_matrix[i][j] = cosine_bw_2_cell_types(adatas_dict_tissue,
                                                     list(adatas_dict_tissue.keys())[i],
                                                     list(adatas_dict_tissue.keys())[j])
            curr_matrix[j][i] = curr_matrix[i][j]
            print('element', i, j, 'done (',list(adatas_dict_tissue.keys())[i],',',list(adatas_dict_tissue.keys())[j],')')
    dataframe = pd.DataFrame(data=curr_matrix, 
                             columns=list(adatas_dict_tissue.keys()),
                             index=list(adatas_dict_tissue.keys()))
    return(dataframe)    
    

In [7]:
def create_df_wasserstein_binned(adatas_dict_tissue):
    curr_matrix = np.zeros((len(adatas_dict_tissue),len(adatas_dict_tissue)))
    for i in range(len(adatas_dict_tissue)):
        for j in range(i):
            curr_matrix[i][j] = wass_bw_2_cell_types_binned(adatas_dict_tissue,
                                                     list(adatas_dict_tissue.keys())[i],
                                                     list(adatas_dict_tissue.keys())[j])
            curr_matrix[j][i] = curr_matrix[i][j]
            print('element', i, j, 'done (',list(adatas_dict_tissue.keys())[i],',',list(adatas_dict_tissue.keys())[j],')')
    dataframe = pd.DataFrame(data=curr_matrix, 
                             columns=list(adatas_dict_tissue.keys()),
                             index=list(adatas_dict_tissue.keys()))
    return(dataframe)    
    

In [8]:
def create_df_pearson(adatas_dict_tissue):
    curr_matrix = np.zeros((len(adatas_dict_tissue),len(adatas_dict_tissue)))
    for i in range(len(adatas_dict_tissue)):
        for j in range(i):
            curr_matrix[i][j] = pearson_bw_2_cell_types(adatas_dict_tissue,
                                                     list(adatas_dict_tissue.keys())[i],
                                                     list(adatas_dict_tissue.keys())[j])[0]
            curr_matrix[j][i] = curr_matrix[i][j]
            print('element', i, j, 'done (',list(adatas_dict_tissue.keys())[i],',',list(adatas_dict_tissue.keys())[j],')')
    dataframe = pd.DataFrame(data=curr_matrix, 
                             columns=list(adatas_dict_tissue.keys()),
                             index=list(adatas_dict_tissue.keys()))
    return(dataframe)    
    

Selecting DE genes

In [9]:
# using logFC > 1 and FDR(p-value) < 5% cutoffs by default
def create_union_list_of_DE_genes(adata, celltype_column, logFC_min = 1.0, FDR_max = 0.05):
    curr_dict = {}
    union_DE_genes = []
    for celltype in list(set(adata.obs[celltype_column])):
        #print(celltype)
        celltype = str(celltype)
        curr_genes = (pd.DataFrame(data=adata.uns['rank_genes_groups']['names']))[celltype]
        curr_log2fold = (pd.DataFrame(data=adata.uns['rank_genes_groups']['logfoldchanges']))[celltype]
        curr_pvals = (pd.DataFrame(data=adata.uns['rank_genes_groups']['pvals']))[celltype]
        curr_dict[celltype] = pd.DataFrame(data={'genes':curr_genes, 'log2foldchange':curr_log2fold, 'pvals':curr_pvals})
        curr_dict[celltype] = curr_dict[celltype][curr_dict[celltype]['log2foldchange'] > logFC_min]
        curr_dict[celltype] = curr_dict[celltype][curr_dict[celltype]['pvals'] < FDR_max]            
        union_DE_genes.append(curr_dict[celltype]['genes'])
    union_DE_genes = list(set([item for sublist in union_DE_genes for item in sublist]))
    return(union_DE_genes)


## Generating distance matrices

### Documentation

sc_to_dist(celltype_column_str, distance_list, name_str, 
                filepath_X_csv=None, filepath_obs_csv=None, filepath_h5ad = None): 
<br>
generates distance matrices from single cell RNA-seq data
<br>
<br>
Input: scRNA-seq dataset, can be provided trough either an h5ad file or separate gene expression and annotation tables (if saved through rds file in R)
<br>
Output: 0 if correct and successful execution (distance matrices are saved in working directory), -1 otherwise
<br>
<br>
Arguments:
<br>
1. celltype_column_str: srt - name of the column containing cell type lables in the annotation table
<br>
2. distance_list: list - list of distance names to be used to generate distance matrices
<br>
    available distances and their names: cosine similarity - input as eiter 'cosine' or 'cosine similarity'; Pearson's correlation - input as eiter 'Pearson's correlation' or 'correlation'; Wasserstein distance - input as either 'Wasserstein' or 'earth mover's distance'
<br>
3. name_str: str - name of the organism or just a code used to name the saved file
<br>
4. filepath_X_csv: optional, None - path to the file with gene expression table in csv format, tab delimited
<br>
5. filepath_obs_csv: optional, None - path to the file with annotation table in csv format, tab delimited
<br>
6. filepath_h5ad: optional, None - path to the h5ad file with scRNA-seq data
<br>
NOTE: out of arguments 4, 5 and 6 you have to provide either 4 and 5 or 6
<br>
7. alpha: 0.8 - gene subsetting parameter, what proportion of genes to take into account for distance matrix calculation


### Alpha - subsetting parameter (how many genes we are tsking from all DE genes)

In [10]:
# 80%
alpha = 0.8

In [11]:
def sc_to_dist_subset(celltype_column_str, distance_list, name_str,
               filepath_X_csv = None, filepath_obs_csv = None, filepath_h5ad = None, alpha=0.8):
    # don't forget to import stuff in the beginning and compile all other general functions
    # add tests/checks
    
    if filepath_h5ad == None:
        curr_X = pd.read_csv(filepath_X_csv, sep='\t')
        curr_obs = pd.read_csv(filepath_obs_csv, sep='\t')
        curr_adata = anndata.AnnData(X=curr_X.T,obs=curr_obs)
        for col in curr_obs.columns:
            curr_adata.obs[col] = curr_obs[col]
        print('adata object created, expression matrix dimensions:', curr_adata.X.shape)
    else:
        curr_adata = sc.read(filepath_h5ad)
        print('adata object created, expression matrix dimensions:', curr_adata.X.shape)
    curr_adata.obs[celltype_column_str] = curr_adata.obs[celltype_column_str].astype('category')
    sc.tl.rank_genes_groups(curr_adata,celltype_column_str,method='wilcoxon')
    DE_genes = create_union_list_of_DE_genes(curr_adata,celltype_column_str)
    
    print('randomly selecting a proportion (alpha) of genes') 
    np.random.shuffle(DE_genes)
    print('list of ', int(alpha*100),'% of all DE genes (', len(DE_genes),' ) created, of total initial', 
          len(list(curr_adata.var_names)), 'genes')
    DE_genes = DE_genes[:round(alpha*len(DE_genes))]
    print('a snippet of DE_genes subset:',DE_genes[:5])
    
    # subsetting the adata object
    curr_adata = curr_adata[:,[i in DE_genes for i in curr_adata.var_names]]
    print('adata object subsetted to DE genes only, dimensions:', curr_adata.X.shape)
    
    curr_adata_dict = create_adatas_dicts(curr_adata, celltype_column_str)
    
    for distance in distance_list:
        if distance == 'cosine' or distance == 'cosine similarity':
            print('creating cosine similarity distance matrix')
            df_DE_cosine = create_df_cosine(curr_adata_dict)
            df_DE_cosine.to_csv(name_str+'_cosine_dist_matrix_DE_subset'+str(alpha)+'.csv',sep='\t')
            print('saved cosine matrix to ',name_str+'_cosine_dist_matrix_DE_subset'+str(alpha)+'.csv')
        elif distance == "Pearson's correlation" or distance == 'correlation':
            print("creating Pearson's correlation distance matrix")
            df_DE_pearson = create_df_pearson(curr_adata_dict)
            df_DE_pearson.to_csv(name_str+'_pearson_dist_matrix_DE_subset'+str(alpha)+'.csv',sep='\t')    
            print("saved Pearson's correlation matrix to ',name_str+'_pearson_dist_matrix_DE_subset"+str(alpha)+".csv")
        elif distance == 'Wasserstein' or distance == "earth mover's distance":
            print("creating Wasserstein (earth mover's) distance distance matrix")            
            df_DE_wasserstein = create_df_wasserstein_binned(curr_adata_dict)
            df_DE_wasserstein.to_csv(name_str+'_wasserstein_dist_matrix_DE_subset'+str(alpha)+'.csv',sep='\t')
            print('saved Wasserstein matrix to ',name_str+'_wasserstein_dist_matrix_DE_subset'+str(alpha)+'.csv')
    return(0)


In [52]:
%%time
# trying out for mouse, cosine
sc_to_dist_subset('celltype',['cosine'],'mouse',filepath_h5ad='/home/jovyan/notebooks/Tree_of_Cells/Mouse_devo.h5ad')

adata object created, expression matrix dimensions: (9227, 29452)
ranking genes
    finished (0:01:04.73) --> added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids
randomly selecting a proportion (alpha) of genes
list of  80 % of all DE genes ( 1153  ) created, of total initial 29452 genes
adata object subsetted to DE genes only, dimensions: (9227, 922)
34 cell types: ['Blood progenitors 2', 'Surface ectoderm', 'Caudal neurectoderm', 'Rostral neurectoderm', 'Neural crest', 'Cardiomyocytes', 'Pharyngeal mesoderm', 'Nascent mesoderm', 'Haematoendothelial progenitors', 'Caudal epiblast', 'Caudal Mesoderm', 'Anterior Primitive Streak', 'Blood progenitors 1', 'Allantois', 'PGC', 'Forebrain/Midbrain/Hindb

0

In [55]:
# trying out 100 times for mouse, cosine
# this took 5-6 hours
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('celltype',['cosine'],'mouse'+str(i+1),filepath_h5ad='/home/jovyan/notebooks/Tree_of_Cells/Mouse_devo.h5ad')




creating distance matrix number 1
adata object created, expression matrix dimensions: (9227, 29452)
ranking genes
    finished (0:01:05.06) --> added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids
randomly selecting a proportion (alpha) of genes
list of  80 % of all DE genes ( 1153  ) created, of total initial 29452 genes
a snippet of DE_genes subset: ['ENSMUSG00000024308', 'ENSMUSG00000089804', 'ENSMUSG00000031633', 'ENSMUSG00000038235', 'ENSMUSG00000021613']
adata object subsetted to DE genes only, dimensions: (9227, 922)
34 cell types: ['Blood progenitors 2', 'Surface ectoderm', 'Caudal neurectoderm', 'Rostral neurectoderm', 'Neural crest', 'Cardiomyocytes', 'Pharyngeal mesoderm', 'Nascent mesod

In [ ]:
%%time
# started at 13:32 18.05.19
# 100 times for planaria, cosine
# now 97 (from i=27 to i=99) because 27 trees we've already created and saved 
for i in range(27,100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('cell_type1',['cosine'],'planaria'+str(i+1),filepath_h5ad='/home/jovyan/notebooks/Tree_of_Cells/Planaria.h5ad')




creating distance matrix number 28
adata object created, expression matrix dimensions: (21612, 28065)
ranking genes
    finished (0:03:42.56) --> added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids
randomly selecting a proportion (alpha) of genes
list of  80 % of all DE genes ( 1629  ) created, of total initial 28065 genes
a snippet of DE_genes subset: ['dd_Smed_v6_1240_0', 'dd_Smed_v6_2024_0', 'dd_Smed_v6_250_0', 'dd_Smed_v6_6287_0', 'dd_Smed_v6_181_0']
adata object subsetted to DE genes only, dimensions: (21612, 1303)
51 cell types: ['parenchymal progenitors', 'secretory 1', 'pigment', 'neural progenitors', 'ChAT neurons 1', 'otf+ cells 2', 'otf+ cells 1', 'muscle progenitors', 'pharynx cell typ

In [ ]:
%%time
# 100 times for Sim_1112, cosine
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('Group',['cosine'],'Sim_1112_'+str(i+1),
                      filepath_X_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_1112_X.csv',
                      filepath_obs_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_1112_obs.csv')




In [ ]:
%%time
# 100 times for Sim_3814, cosine
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('Group',['cosine'],'Sim_3814_'+str(i+1),
                      filepath_X_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_3814_X.csv',
                      filepath_obs_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_3814_obs.csv')




In [ ]:
%%time
# 100 times for Sim_528, cosine
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('Group',['cosine'],'Sim_528_'+str(i+1),
                      filepath_X_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_528_X.csv',
                      filepath_obs_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_528_obs.csv')




In [ ]:
%%time
# 100 times for Sim_71640, cosine
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('Group',['cosine'],'Sim_71640_'+str(i+1),
                      filepath_X_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_71640_X.csv',
                      filepath_obs_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_71640_obs.csv')




In [ ]:
%%time
# 100 times for Sim_9901, cosine
for i in range(100):
    print('creating distance matrix number',i+1)
    sc_to_dist_subset('Group',['cosine'],'Sim_9901_'+str(i+1),
                      filepath_X_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_9901_X.csv',
                      filepath_obs_csv = '/home/jovyan/notebooks/Tree_of_Cells/Simulations_Tallulah/Sim_9901_obs.csv')


